# Plotly examples

In [ ]:
import numpy as np
import plotly.graph_objs as go

In [ ]:
from plotly.subplots import make_subplots

N = 50
x = np.arange(N)
y = np.arange(N)
z = np.random.rand(N, N)

fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Scatter(x=x, y=z[:, 0]), row=1, col=1)

fig.add_trace(go.Histogram(x=z[:, 0], xbins={"size": 0.05}), row=1, col=2)

fig.add_trace(go.Heatmap(z=z, showscale=False), row=2, col=1)

fig.add_trace(go.Scatter(x=z[:, 0], y=z[:, 1], mode='markers',
    marker={"color": z[:, 2]}), row=2, col=2)

fig.show()

## 1. Scrolling with the mouse through the 3rd dimension of a 3D data cube

https://matplotlib.org/examples/pylab_examples/image_slices_viewer.html

In [ ]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = np.clip(self.ind + 1, 0, self.slices - 1)
        else:
            self.ind = np.clip(self.ind - 1, 0, self.slices - 1)
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()


fig, ax = plt.subplots(1, 1)

X = np.random.rand(20, 20, 40)

tracker = IndexTracker(ax, X)


fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

## 2. Click on a legend item to hide/show the corresponding line

https://matplotlib.org/examples/event_handling/legend_picking.html

<a id='plotly_in_qt'></a>
## Embed plotly graph in Qt window

In [ ]:
import os, sys
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5 import QtWebEngineWidgets
import plotly.io as pio
import plotly.offline as offplt
import plotly.graph_objs as go


class PlotlyViewer(QtWebEngineWidgets.QWebEngineView):
    def __init__(self, fig, exec=True):
        # Create a QApplication instance or use the existing one if it exists
        self.app = QApplication.instance() if QApplication.instance() else QApplication(sys.argv)

        super().__init__()

        self.file_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "temp.html"))
        html = offplt.plot(fig, output_type='', auto_open=False, include_plotlyjs="True")
        print(html)
        self.setHtml(html)
        self.setWindowTitle("Plotly Viewer")
        self.show()

        if exec:
            self.app.exec_()

    def closeEvent(self, event):
        os.remove(self.file_path)


# Or a custom external renderer
class MyExternalRenderer(pio.base_renderers.ExternalRenderer):
    def activate(self):
        pass

    def render(self, fig_dict):
        for key, fig in fig_dict.items():
            win = PlotlyViewer(fig)


# Register for use by name
my_renderer = MyExternalRenderer()
pio.renderers['my_renderer'] = my_renderer

fig = go.Figure()
fig.add_scatter(x=np.random.rand(100), y=np.random.rand(100), mode='markers',
                marker={'size': 30, 'color': np.random.rand(100), 'opacity': 0.6,
                        'colorscale': 'Viridis'})
fig.show(renderer='my_renderer')